In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.systems.sivep import Sivep

In [ ]:
def generate_table(sivep_df):
    tabela = sivep_df.groupby(['SE','CLASSI_FIN'])[['SE']].count().unstack()
    
    tabela = tabela.droplevel(0, axis=1)
    
    tabela['1'] = tabela['1'].add(tabela['2'])
    tabela['1'] = tabela['1'].add(tabela['3'])
    tabela = tabela[['5', '1', '4', '0']]
    
    tabela = tabela.rename(columns={'5':'Casos confirmados', '1':'Casos descartados', '4':'Casos suspeitos', '0':'Não informados'})
    
    
    tabela['Total testes'] = tabela.sum(1)
    tabela = tabela.append(pd.DataFrame(tabela.sum(0)).transpose())
    tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])
    
    tabela.index.name = 'SE'
    tabela.iloc[-1].name = 'Total Geral'
    
#     tabela.to_excel(join(default_input,'semanais','TABELA_sivep_INPUT.xlsx'))
#     tabela.to_excel('output/TABELA_testes_sivep_RS.xlsx')
    tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values[:-1]] + ['Total Geral']
    tabela = tabela.set_index('Semana Epidemiológica')
    
    tabela.columns.name = None
    tabela.reset_index(inplace=True)
    
    for j in tabela.columns:
        tabela[j] = tabela[j].fillna(0)
    
    return tabela

In [ ]:
#12ª RS - Umuarama
# list_IBGE_RS = ['412862','410070','410050','410337','410347','410660','410725','410752','410832','410990','411060','411155','411470','411510','411720','411885','411890','412535','412690','412810','412880']

# exames_RS = pd.DataFrame()
# for municipio in list_IBGE_RS:
#     exames_temp = sivep_df.loc[sivep_df['CO_MUN_NOT'] == municipio]
#     exames_RS = exames_RS.append(exames_temp)
# sivep_df = exames_RS

In [ ]:
sivep = Sivep()
# sivep.load()
try:
    sivep.read_all_database_files_dbf()
except:
    sivep.read_all_database_files_excel()
sivep.save(replace=True)

In [ ]:
sivep_df = sivep.df

In [ ]:
sivep_df = sivep_df.loc[sivep_df['SG_UF'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['SG_UF_NOT'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['AMOSTRA'] == '1']
sivep_df.loc[sivep_df['CLASSI_FIN'].isnull(), 'CLASSI_FIN'] = '0'

sivep_df['NU_IDADE_N'] = sivep_df['NU_IDADE_N'].fillna('0').astype(int)
sivep_df.loc[sivep_df['TP_IDADE'].isin(['1', '2']), ('TP_IDADE','NU_IDADE_N')] = ('3', 0) #dia, mes

In [ ]:
sivep_df['DT_NOTIFIC'] = sivep_df['DT_NOTIFIC'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
sivep_df['SE'] = sivep_df['DT_NOTIFIC'].apply(lambda x: Week.fromdate(x))

In [ ]:
geral = generate_table(sivep_df)
geral.to_excel(join(default_output, 'semanais','TABELA_testes_sivep.xlsx'), index=False)
geral

In [ ]:
grupo01 = sivep_df.loc[(sivep_df['NU_IDADE_N'] >= 0) & (sivep_df['NU_IDADE_N'] <= 9)]

tabela_grupo01 = generate_table(grupo01)
tabela_grupo01.to_excel(join(default_output, 'semanais','TABELA_testes_sivep_GRUPO01.xlsx'), index=False)
tabela_grupo01

In [ ]:
grupo02 = sivep_df.loc[(sivep_df['NU_IDADE_N'] >= 10) & (sivep_df['NU_IDADE_N'] <= 19)]

tabela_grupo02 = generate_table(grupo02)
tabela_grupo02.to_excel(join(default_output, 'semanais','TABELA_testes_sivep_GRUPO02.xlsx'), index=False)
tabela_grupo02